In [2]:
import os
import re
import json
import subprocess

from openai import OpenAI
import sys
sys.path.append('../')
from prompts.design_prompt import design_prompt
from prompts.parsing_prompt import parsing_prompt
from prompts.scripting_prompt import scripting_prompt

In [4]:
with open('../config/config.json') as f:
     config = json.load(f)

In [5]:
OPENSCAD_EXEC = config["OPENSCAD_EXEC_PATH"]
FILE_DIR = "../common/spur_gear"
DESIGN_FILENAME = "gears_design.md"
PARAMETERS_FILENAME = "gears_parameters.json"
SCRIPT_FILENAME = "gears_script.scad"
if not os.path.exists(FILE_DIR):
    os.makedirs(FILE_DIR)

In [5]:
client = OpenAI(api_key=config['OPENAI_API_KEY'])

In [5]:
design_description = f""" 
Suppose there is a one-stage system with two spur gears.
Gear axes are parallel and the distance between two axes is 100mm. 
The height of driven and driving gears are 27mm and 18mm respectively, the pressure angle are both 20 degrees.
The desired gear ratio of the reduction system is 3:1.
Set the driving gear to the center of the plane, which is the coordinate (0, 0)
Based on the given information, computer module number, teeth number of both gears and the coordniate of the driven gear centers.
"""

In [9]:
design_messages=[
        {"role": "system", "content": design_prompt},
        {"role": "user", "content": design_description},
    ]

In [8]:
# stop_tokens = ['']
# collected_tokens = []

# res = client.chat.completions.create(
#     model=config['GPT_MODEL'],
#     messages=messages,
#     stream=True,
# )

# for chunk in res:
#     temp_tokens = chunk.choices[0].delta.content
#     if temp_tokens not in stop_tokens:
#         collected_tokens.append(temp_tokens)
#     else:
#         chunk_res = "".join(collected_tokens).strip()
#         print(chunk_res)
#         collected_tokens = []

In [16]:
res = client.chat.completions.create(
    model=config['GPT_MODEL'],
    messages=design_messages,
    temperature=0.1
)

In [17]:
print(res.choices[0].message.content)

To design the gear system, we need to determine the module, number of teeth for both gears, and the coordinates of the driven gear center. Let's start by calculating the module and number of teeth for both gears to achieve the desired gear ratio of 3:1.

### Step 1: Gear Ratio and Number of Teeth
The gear ratio (i) is defined as the ratio of the number of teeth on the driven gear (N2) to the number of teeth on the driving gear (N1):

$$ i = \frac{N2}{N1} $$

Given the desired gear ratio is 3:1, we can express N2 as 3 times N1:

$$ N2 = 3N1 $$

### Step 2: Module Calculation
The module (m) is a measure of the size of the teeth and is related to the pitch diameter (d) and the number of teeth (N) by the formula:

$$ m = \frac{d}{N} $$

The center distance (a) between the two gears can be expressed in terms of the module and the number of teeth:

$$ a = \frac{m(N1 + N2)}{2} $$

Given that the distance between the gear axes is 100mm, we can set this equal to the center distance:

$$ 100mm =

In [18]:
with open(os.path.join(FILE_DIR, DESIGN_FILENAME), "w") as f:
    f.write(res.choices[0].message.content)

In [19]:
parsing_description = \
f"gear 1 (driving gear): spur gear, module 2.5mm, teeth 20, pitch diameter 50mm, height 18mm, coordinate (0,0)" + \
f"pressure angle 20 degrees \n" + \
f"gear 2 (driven gear): spur gear, module 2.5mm, teeth 60, pitch diameter 150mm, height 27mm, coordinate (100,0)" + \
f"pressure angle 20 degrees"

In [20]:
parsing_messages = [
    {"role": "system", "content": parsing_prompt},
    {"role": "user", "content": parsing_description},
]

In [21]:
parsing_response = client.chat.completions.create(
    model=config['GPT_MODEL'],
    response_format={ "type": "json_object" },
    messages=parsing_messages,
)

In [22]:
gears_parameters = parsing_response.choices[0].message.content.strip()
print(gears_parameters)

{
  "gear_1": {
    "gear_type": "spur",
    "source": "driving",
    "coordinate": {
      "x": 0,
      "y": 0,
      "z": 0,
      "alpha": 0,
      "beta": 0,
      "gamma": 0
    },
    "unit": "mm",
    "module": 2.5,
    "teeth": 20,
    "height": 18,
    "pitch_d": 50,
    "helix_angle": 0,
    "pitch_angle": 0,
    "pressure_angle": 20
  },
  "gear_2": {
    "gear_type": "spur",
    "source": "driven",
    "coordinate": {
      "x": 100,
      "y": 0,
      "z": 0,
      "alpha": 0,
      "beta": 0,
      "gamma": 0
    },
    "unit": "mm",
    "module": 2.5,
    "teeth": 60,
    "height": 27,
    "pitch_d": 150,
    "helix_angle": 0,
    "pitch_angle": 0,
    "pressure_angle": 20
  }
}


In [25]:
with open(os.path.join(FILE_DIR, PARAMETERS_FILENAME), "w") as f:
    f.write(gears_parameters)

In [26]:
with open(os.path.join(FILE_DIR, PARAMETERS_FILENAME), "r") as f:
    parameters = json.load(f)
parameters = str(parameters)

scripting_description = f"Write a script according to provided parameters of gears \n" + parameters

In [27]:
script_messages = [
    {"role": "system", "content": scripting_prompt},
    {"role": "user", "content": scripting_description}
]

In [54]:
with open("../common/spur_gear/gears_parameter.json", "r") as f:
    parameters = json.load(f)
parameters = str(parameters)

scripting_description = f"Write a script according to provided procedure with parameters of gears" + parameters

In [33]:
print(scripting_description)

Write a script according to provided parameters of gears 
{'gear_1': {'gear_type': 'spur', 'source': 'driving', 'coordinate': {'x': 0, 'y': 0, 'z': 0, 'alpha': 0, 'beta': 0, 'gamma': 0}, 'unit': 'mm', 'module': 2.5, 'teeth': 20, 'height': 18, 'pitch_d': 50, 'helix_angle': 0, 'pitch_angle': 0, 'pressure_angle': 20}, 'gear_2': {'gear_type': 'spur', 'source': 'driven', 'coordinate': {'x': 100, 'y': 0, 'z': 0, 'alpha': 0, 'beta': 0, 'gamma': 0}, 'unit': 'mm', 'module': 2.5, 'teeth': 60, 'height': 27, 'pitch_d': 150, 'helix_angle': 0, 'pitch_angle': 0, 'pressure_angle': 20}}


In [55]:
script_messages = [
    {"role": "system", "content": scripting_prompt},
    {"role": "user", "content": scripting_description}
]

In [28]:
scripting_response = client.chat.completions.create(
    model=config['GPT_MODEL'],
    messages=script_messages,
)

In [29]:
def script_postprocessing(script):
    # To remove textual perfix or suffix information in the raw script generated by GPT-4
    pattern = re.compile(r'```(?:openscad|scad)(.*?)```', re.DOTALL)
    matches = re.findall(pattern, script)
    clean_script = ""
    if len(matches) > 0:
        for m in matches:
            clean_script = clean_script + m.strip()
    else:
        clean_script = script
    return clean_script

In [30]:
script = scripting_response.choices[0].message.content.strip()
script = script_postprocessing(script)
print(script)

translate([0,0,0]) rotate([0,0,0])
gear(m=2.5, z=20, h=18, w=20);

translate([100,0,0]) rotate([0,0,(360/(60*2))])
gear(m=2.5, z=60, h=27, w=20);


In [31]:
with open(os.path.join(FILE_DIR, SCRIPT_FILENAME), "w") as f:
    f.write("include <gears.scad>")

with open(os.path.join(FILE_DIR, SCRIPT_FILENAME), "a") as f:
    f.write("\n" + script)

In [6]:
subprocess.Popen([OPENSCAD_EXEC, os.path.join(FILE_DIR, SCRIPT_FILENAME)])

<Popen: returncode: None args: ['D:\\openSCAD\\install\\openscad.exe', '../c...>